In [1]:
import requests
import pandas as pd

API_REQUEST_URL = "https://api.scb.se/OV0104/v1/doris/en/ssd/START/MI/MI0108/TotaltUtslapp"
API_REQUEST_TEMPLATE = {
    "query": [
        {
            "code": "Sektor",
            "selection": {
                "filter": "item",
                "values": ["0.6"]  # Selecting sector "0.6"
            }
        }
    ],
    "response": {
        "format": "json"  # Requesting JSON format
    }
}

# API request
response = requests.post(API_REQUEST_URL, json=API_REQUEST_TEMPLATE)

if response.status_code == 200:
    data = response.json()
    
    # put data in dataframe (there's metadata, data, etc.)
    emissions_data = pd.DataFrame(data["data"])
    
    # show how `key` and `values` look
    print("Sample data:\n", emissions_data.head())

    # get pollutant, sector, and year from `key`
    emissions_data[['Pollutant', 'Sector', 'Year']] = pd.DataFrame(emissions_data["key"].apply(lambda x: x).tolist(), index=emissions_data.index)

    emissions_data['Year'] = pd.to_numeric(emissions_data['Year'], errors='coerce')
    
    # get emissions specifically
    emissions_data['Emissions'] = emissions_data['values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    # clean and show
    sweden_aq = emissions_data[['Year', 'Pollutant', 'Sector', 'Emissions']]
    print(sweden_aq.head())
else:
    print(f"Error: {response.status_code}")

# save as csv file
sweden_aq.to_csv('/Users/sarahkilpatrick/Documents/CSDE 533 Homeworks/sweden_aq.csv', index=False)


Sample data:
                 key      values
0  [NOx, 0.6, 1990]  [339924.9]
1  [NOx, 0.6, 1991]  [351223.8]
2  [NOx, 0.6, 1992]  [343497.7]
3  [NOx, 0.6, 1993]  [329671.6]
4  [NOx, 0.6, 1994]  [344226.4]
   Year Pollutant Sector Emissions
0  1990       NOx    0.6  339924.9
1  1991       NOx    0.6  351223.8
2  1992       NOx    0.6  343497.7
3  1993       NOx    0.6  329671.6
4  1994       NOx    0.6  344226.4
Data saved as 'sweden_aq.csv'
